In [7]:
# for making cif_path_list
import time
from pathlib import Path
import re
from tqdm import tqdm

def get_subdir_list(p_sub_list):
    """
    To get a sub directory path list, Use thie func().
    
    pram: p_aub_list: specify a directory which sub dirs is gotten from.
    """
    # 引数の直下のディレクトリ・パスの一覧を取得
    sub_dir_list_temp = []
    for p_sub in tqdm(p_sub_list):
        sub_dir_list_temp.append([p_s_s for p_s_s in p_sub.iterdir()])
    # ２次元リストを１次元リスト化
    return sum(sub_dir_list_temp, [])


p = Path('../cif/')
p_sub_list = [p_s for p_s in p.glob('[0-9]')]


before = time.time()
cif_path_list = get_subdir_list(get_subdir_list(get_subdir_list(p_sub_list)))
after = time.time()
print(f"it took {after - before}sec to get all of subdir list.")


def cif_filter(cif_file_path):
    pattern = '.*\.cif'
    cif_file_path_str = str(cif_file_path)
    return bool(re.match(pattern, cif_file_path_str))


cif_path_list = [str(path) for path in cif_path_list if cif_filter(path)]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6898/6898 [00:48<00:00, 142.95it/s]


it took 110.62443852424622sec to get all of subdir list.


### make poscar file by cif2cell

In [1]:
!pip install git+https://github.com/torbjornbjorkman/cif2cell.git

  Cloning https://github.com/torbjornbjorkman/cif2cell.git to /tmp/pip-req-build-9sdtwubn
  Running command git clone -q https://github.com/torbjornbjorkman/cif2cell.git /tmp/pip-req-build-9sdtwubn
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.1 MB 1.8 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 49 kB 1.7 MB/s eta 0:00:01
  Created wheel for cif2cell: filename=cif2cell-2.1.0-py3-none-any.whl size=2060209 sha256=d29ad5522b784fa81719ff9f38f6dd42c32f2f9312552fc62eaae0f8400a6c9e
  Stored in directory: /tmp/pip-ephem-wheel-cache-d3j8ojam/wheels/15/d3/60/346b10b0ab63b24b8d607b84d03734de09f0056e063edb7e21
  Created wheel for PyCifRW: filename=PyCifRW-4.4.6-cp38-cp38-linux_x86_64.whl size=165885

シェルのコマンド:\
!cif2cell ../cif/1/00/00/1000000.cif -p vasp --vasp-format=5 -o ../cif/1/00/00/1000000/POSCAR

In [16]:
import subprocess

cif2cellの使い方：\
$ cif2cell ABW.cif -p vasp --vasp-format=5

In [23]:
cif_file = str(cif_path_list[0])
print(cif_file)

../cif/1/00/00/1000000.cif


In [26]:
cif_file = str(cif_path_list[0])
subprocess.run(['cif2cell', cif_file, '-p vasp', '--vasp-format=5'])

Error: Unknown output format:  vasp


CompletedProcess(args=['cif2cell', '../cif/1/00/00/1000000.cif', '-p vasp', '--vasp-format=5'], returncode=1)

In [61]:
cif_file[0:-4] + '/POSCAR'

'../cif/1/00/00/1000000/POSCAR'

In [72]:
# Success example
cif_file = str(cif_path_list[0])
poscar_file_path = cif_file[0:-4] + '/POSCAR'
subprocess.run(['cif2cell', cif_file, '-p', 'vasp', '-o', poscar_file_path, '--vasp-format=5'])

CompletedProcess(args=['cif2cell', '../cif/1/00/00/1000000.cif', '-p', 'vasp', '-o', '../cif/1/00/00/1000000/POSCAR', '--vasp-format=5'], returncode=0)

#### confirmation by using shell command

In [78]:
!rm ../cif/1/00/00/1000000/POSCAR

In [42]:
# !cif2cell ../cif/1/00/00/1000000.cif -p vasp --vasp-format=5 -o ../cif/1/00/00/1000000/POSCAR

In [79]:
!ls ../cif/1/00/00/1000000

In [80]:
!cat ../cif/1/00/00/1000000/POSCAR

cat: ../cif/1/00/00/1000000/POSCAR: No such file or directory


### cif2cellを約54万件のデータに適用

In [99]:
cif_path_list = [str(path) for path in cif_path_list if cif_filter(path)]
poscar_path_list = [str(cif_path)[0:-4] + '/POSCAR' for cif_path in cif_path_list]

In [124]:
from tqdm.notebook import tqdm
for cif_path, poscar_path in tqdm(zip(cif_path_list[0:5], poscar_path_list[0:5]), total=len(cif_path_list)):
    error_list = []
    try:
        subprocess.run(['cif2cell', cif_path, '-p', 'vasp', '-o', poscar_path, '--vasp-format=5'])
    except Exception as e:
        error_list.appens(e)

  0%|          | 0/504259 [00:00<?, ?it/s]

In [140]:
!pwd

/mnt/ssd_elecom_red/scripts
